# HuggingFace 커스텀 프로젝트 만들기

## 1. 모듈 import

In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
import os,sys,copy,time
import urllib

# Hugging face
import transformers
from transformers import AutoModel, AutoTokenizer
from transformers import AutoTokenizer, AutoModelForSequenceClassification, DataCollatorWithPadding
import datasets
from datasets import load_dataset

In [2]:
train1 = os.getenv('HOME')+'/aiffel/ratings_train.txt'
test1 = os.getenv('HOME')+'/aiffel/ratings_test.txt'

In [3]:
train = pd.read_table(train1)
test = pd.read_table(test1)

In [4]:
train.head()

,id,document,label
0,9976970,아 더빙.. 진짜 짜증나네요 목소리,0
1,3819312,흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나,1
2,10265843,너무재밓었다그래서보는것을추천한다,0
3,9045019,교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정,0
4,6483659,사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...,1


In [5]:
huggingface_nsmc_dataset = load_dataset('nsmc')
print(huggingface_nsmc_dataset)

Using custom data configuration default
Reusing dataset nsmc (/aiffel/.cache/huggingface/datasets/nsmc/default/1.1.0/bfd4729bf1a67114e5267e6916b9e4807010aeb238e4a3c2b95fbfa3a014b5f3)


  0%|          | 0/2 [00:00<?, ?it/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 150000
    })
    test: Dataset({
        features: ['id', 'document', 'label'],
        num_rows: 50000
    })
})


## 2. 토크나이저 생성

In [6]:
tokenizer = AutoTokenizer.from_pretrained("klue/bert-base")
model = AutoModelForSequenceClassification.from_pretrained("klue/bert-base", num_labels = 2)

Some weights of the model checkpoint at klue/bert-base were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized

In [7]:
def transform(data):
  return tokenizer(
      data['document'],
      truncation = True,
      #padding = 'longest', #'max_length','longest', #'max_length',
      return_token_type_ids = False,
      )
  
examples = huggingface_nsmc_dataset['train'][:2]
examples_transformed = transform(examples)

print(examples)
print(examples_transformed)

{'id': ['9976970', '3819312'], 'document': ['아 더빙.. 진짜 짜증나네요 목소리', '흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나'], 'label': [0, 1]}
{'input_ids': [[2, 1376, 831, 2604, 18, 18, 4229, 9801, 2075, 2203, 2182, 4243, 3], [2, 1963, 18, 18, 18, 11811, 2178, 2088, 28883, 16516, 2776, 18, 18, 18, 18, 10737, 2156, 2015, 2446, 2232, 6758, 2118, 1380, 6074, 3]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [8]:
# 데이터 토크나이징 완료
encoded_dataset = huggingface_nsmc_dataset.map(transform, batched=True)

# 동적패딩: Bucketing 
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

  0%|          | 0/150 [00:00<?, ?ba/s]

  0%|          | 0/50 [00:00<?, ?ba/s]

In [9]:
from datasets import load_metric
metric = load_metric('glue', 'qnli')

def compute_metrics(eval_pred):    
    predictions,labels = eval_pred
    print(predictions.shape, predictions)
    predictions = np.argmax(predictions, axis=1)
    print(predictions.shape, predictions)
    return metric.compute(predictions=predictions, references = labels)

## 3. 훈련 및 평가

In [10]:
# TrainingArguments 생성 : Trainer을 활용하는 형태로 모델 재생성
from transformers import Trainer, TrainingArguments
output_dir = os.getenv('HOME')+'/aiffel/hugging_face_transformers/output'
metric_name = 'accuracy'

training_arguments = TrainingArguments(
    output_dir, # output이 저장될 경로
    evaluation_strategy="steps",              # "epoch", #evaluation하는 빈도
    learning_rate = 0.001,                   #2e-5, #learning_rate   1e-3
    per_device_train_batch_size = 512,      #64, #64, #16, # 각 device 당 batch size    512
    per_device_eval_batch_size = 512,      #64, #64, #16, # evaluation 시에 batch size   512
    num_train_epochs = 2,                  # train 시킬 총 epochs   20
    weight_decay = 0.01,                   # weight decay
    save_strategy="epoch",                 # 저장은 epoch 마다
    metric_for_best_model=metric_name,  
)

In [11]:
trainer = Trainer(
    model= model,                           # 학습시킬 model
    args=training_arguments,                  # TrainingArguments을 통해 설정한 arguments
    train_dataset=encoded_dataset['train'],    # training dataset
    eval_dataset=encoded_dataset['test'],       # test dataset
    compute_metrics=compute_metrics,
    data_collator = data_collator,
    tokenizer = tokenizer,
)

In [12]:
trainer.evaluate(encoded_dataset['test'])

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 512


(50000, 2) [[ 0.3536646   0.13695504]
 [ 0.3748738  -0.00793599]
 [ 0.4523907  -0.15237457]
 ...
 [-0.06852636  0.20821825]
 [ 0.37208036  0.02408335]
 [ 0.38024706  0.29182684]]
(50000,) [0 0 0 ... 1 0 0]


{'eval_loss': 0.6977577209472656,
 'eval_accuracy': 0.51222,
 'eval_runtime': 301.3266,
 'eval_samples_per_second': 165.933,
 'eval_steps_per_second': 0.325}

In [13]:
# TrainingArguments 생성 : Trainer을 활용하는 형태로 모델 재생성
from transformers import Trainer, TrainingArguments
output_dir = os.getenv('HOME')+'/aiffel/hugging_face_transformers/output'
metric_name = 'accuracy'

training_arguments = TrainingArguments(
    output_dir, # output이 저장될 경로
    evaluation_strategy="steps",              # "epoch", #evaluation하는 빈도
    learning_rate = 2e-5,                    #learning_rate   1e-3
    per_device_train_batch_size = 16,      #64, #64, #16, # 각 device 당 batch size    512
    per_device_eval_batch_size = 16,      #64, #64, #16, # evaluation 시에 batch size   512
    num_train_epochs = 2,                  # train 시킬 총 epochs   20
    weight_decay = 0.01,                   # weight decay
    save_strategy="epoch",                 # 저장은 epoch 마다
    metric_for_best_model=metric_name,  
)

using `logging_steps` to initialize `eval_steps` to 500
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [14]:
trainer = Trainer(
    model= model,                           # 학습시킬 model
    args=training_arguments,                  # TrainingArguments을 통해 설정한 arguments
    train_dataset=encoded_dataset['train'],    # training dataset
    eval_dataset=encoded_dataset['test'],       # test dataset
    compute_metrics=compute_metrics,
    data_collator = data_collator,
    tokenizer = tokenizer,
)

In [15]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running training *****
  Num examples = 150000
  Num Epochs = 2
  Instantaneous batch size per device = 16
  Total train batch size (w. parallel, distributed & accumulation) = 16
  Gradient Accumulation steps = 1
  Total optimization steps = 18750


Step,Training Loss,Validation Loss,Accuracy
500,0.382300,0.316742,0.865000
1000,0.316600,0.298721,0.875000
1500,0.300200,0.347037,0.866260
2000,0.284400,0.288353,0.881920
2500,0.293100,0.291645,0.876720
3000,0.283900,0.267559,0.888980
3500,0.278000,0.321031,0.876940
4000,0.280700,0.279730,0.887620
4500,0.274100,0.277605,0.886080
5000,0.261300,0.264133,0.896860


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 16


(50000, 2) [[-1.3226981   1.5576096 ]
 [ 0.47015467  0.12922174]
 [-0.16868162 -0.86988676]
 ...
 [ 0.8387326  -1.1134475 ]
 [ 2.4216716  -2.767542  ]
 [ 1.3505646  -1.2666966 ]]
(50000,) [1 0 0 ... 0 0 0]


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 16


...

(50000, 2) [[-2.1263819   2.7993166 ]
 [-0.33590072  1.1990703 ]
 [ 1.7375814  -2.6101975 ]
 ...
 [ 0.5445172  -1.3227887 ]
 [ 3.377713   -3.902636  ]
 [ 1.2473344  -1.6138761 ]]
(50000,) [1 1 0 ... 0 0 0]


The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 16


(50000, 2) [[-2.112761    2.7836838 ]
 [-0.40424395  1.2812573 ]
 [ 1.6378759  -2.5063622 ]
 ...
 [ 0.41083452 -1.167566  ]
 [ 3.3316545  -3.8665137 ]
 [ 1.1358777  -1.4947085 ]]
(50000,) [1 1 0 ... 0 0 0]


Saving model checkpoint to /aiffel/aiffel/hugging_face_transformers/output/checkpoint-18750
Configuration saved in /aiffel/aiffel/hugging_face_transformers/output/checkpoint-18750/config.json
Model weights saved in /aiffel/aiffel/hugging_face_transformers/output/checkpoint-18750/pytorch_model.bin
tokenizer config file saved in /aiffel/aiffel/hugging_face_transformers/output/checkpoint-18750/tokenizer_config.json
Special tokens file saved in /aiffel/aiffel/hugging_face_transformers/output/checkpoint-18750/special_tokens_map.json


Training completed. Do not forget to share your model on huggingface.co/models =)




TrainOutput(global_step=18750, training_loss=0.22986080993652344, metrics={'train_runtime': 11971.6652, 'train_samples_per_second': 25.059, 'train_steps_per_second': 1.566, 'total_flos': 9733020604318080.0, 'train_loss': 0.22986080993652344, 'epoch': 2.0})

In [16]:
trainer.evaluate(encoded_dataset['test'])

The following columns in the evaluation set  don't have a corresponding argument in `BertForSequenceClassification.forward` and have been ignored: document, id.
***** Running Evaluation *****
  Num examples = 50000
  Batch size = 16


(50000, 2) [[-2.0881035   2.7594452 ]
 [-0.3890107   1.2618679 ]
 [ 1.6213051  -2.4911911 ]
 ...
 [ 0.42239082 -1.1803601 ]
 [ 3.320626   -3.860013  ]
 [ 1.1174031  -1.4742037 ]]
(50000,) [1 1 0 ... 0 0 0]


{'eval_loss': 0.2848275899887085,
 'eval_accuracy': 0.90676,
 'eval_runtime': 216.6969,
 'eval_samples_per_second': 230.737,
 'eval_steps_per_second': 14.421,
 'epoch': 2.0}

## 4. 결론

Hugging Face 의 transformers를 사용하여, klue/bert_base model을 통해, NSMC 데이터를 Positive, Negative 감성분석을 실행해 보면서,Hugging Face 사용방법을 익혀보는 프로젝트

0.90676의 정학도를 보여줬다.